In [36]:
import glob 
import pandas as pd
import os

filenames = glob.glob('../backend/data/cleaned/*.csv')

arr = [] 

for idx, fname in enumerate(filenames) :
    full_path = os.path.abspath(fname)
    df = pd.read_csv(full_path)
    arr.append(df)
    print(f"File {idx} : {fname} has been read successfully")



print(arr[2])


# for "Year" and "Code" Columns
# if float convert to int
# if string convert to int
for df in arr :
    
    for col in df.columns :
        if col == "Year":
            if df[col].dtype == "float64" :
                df[col] = df[col].astype(int)
            elif df[col].dtype == "object" :
                df[col] = df[col].astype(int)
                
    df.columns = df.columns.str.strip()
    
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].str.strip()
    
                
                
    



merged_df = arr[0]

index = 0 
for df in arr[1:] :
    print(merged_df.dtypes, 'df.dtypes')
    print("------------------------------------------")
    merged_df = pd.merge(merged_df, df, on=['Year', 'Code'], how = 'outer')
    
    # print(merged_df)
    index+= 1

merged_df = merged_df.drop_duplicates()



File 0 : ../backend/data/cleaned\cleaned_pct_renewable_energy.csv has been read successfully
File 1 : ../backend/data/cleaned\cleaned_renewable_output_gwh.csv has been read successfully
File 2 : ../backend/data/cleaned\country_co2_emissions.csv has been read successfully
File 3 : ../backend/data/cleaned\country_co2_emissions_fossil_land.csv has been read successfully
File 4 : ../backend/data/cleaned\emissions_merged_no2_ch4_.csv has been read successfully
File 5 : ../backend/data/cleaned\merged.csv has been read successfully
            Entity  Code   Year   Annual CO2 emissions
0      Afghanistan   AFG   1949                14656.0
1      Afghanistan   AFG   1950                84272.0
2      Afghanistan   AFG   1951                91600.0
3      Afghanistan   AFG   1952                91600.0
4      Afghanistan   AFG   1953               106256.0
...            ...   ...    ...                    ...
23860     Zimbabwe   ZWE   2018             10714598.0
23861     Zimbabwe   ZWE   20

In [39]:
merged_df

,Entity_x,Code,Year,Renewable_Share,Country,Renewable_Output_GWh,Entity_y,Annual CO2 emissions,Entity,Annual CO2 emissions from land-use change,...,land_use_change_co2_per_capita_y,co2_y,methane_y,nitrous_oxide_y,total_ghg.1_y,Country Name_y,Country Code_y,Indicator Name_y,Indicator Code_y,emissions_y
0,NaN,AND,1750,NaN,NaN,NaN,Andorra,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AUS,1750,NaN,NaN,NaN,Australia,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,BRN,1750,NaN,NaN,NaN,Brunei,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,BTN,1750,NaN,NaN,NaN,Bhutan,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,CIV,1750,NaN,NaN,NaN,Cote d'Ivoire,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74366,NaN,WSM,2022,NaN,NaN,NaN,Samoa,249491.0,Samoa,-159493.92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74368,NaN,YEM,2022,NaN,NaN,NaN,Yemen,11356347.0,Yemen,-2037220.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74370,NaN,ZAF,2022,NaN,NaN,NaN,South Africa,404053980.0,South Africa,1126277.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74372,NaN,ZMB,2022,NaN,NaN,NaN,Zambia,8921889.0,Zambia,61071740.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
merged_df.to_csv('output/merged_all.csv', index=False)

: 